In [1]:
%load_ext gprof2dot_magic
from sklearn import datasets, linear_model, neighbors, svm, ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from base import SuperLearner, BMA, try_super_learners, gbm_summary
import pandas as pd
import numpy as np
from scipy import stats
from pyearth import Earth
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # warnings from py-earth

v_folds = 5
seed = 123

ols = linear_model.LinearRegression()
elnet = linear_model.ElasticNetCV(l1_ratio=0.5, cv=v_folds, normalize=True)
ridge = linear_model.RidgeCV(cv=v_folds)
lars = linear_model.LarsCV(cv=v_folds, normalize=True)
lasso = linear_model.LassoCV(cv=v_folds, normalize=True)
nn = neighbors.KNeighborsRegressor(weights='distance')
svm1 = svm.SVR(kernel='linear', C=10, gamma='auto')
svm2 = svm.SVR(kernel='poly', C=10, gamma='auto')
rf = ensemble.RandomForestRegressor(n_estimators=500, max_depth=7,
                                    min_samples_split=5, random_state=seed)
gbm = ensemble.GradientBoostingRegressor()
gbm_cv = GridSearchCV(ensemble.GradientBoostingRegressor(), 
                   param_grid={'learning_rate':[0.005,0.01],
                              'n_estimators':[1000,3000],
                              'max_depth':[3,5]})
# earth is sort of like D/S/A?
earth=Earth(max_terms=50,max_degree=3,use_fast=True,verbose=0) # get this from https://github.com/scikit-learn-contrib/py-earth
rtree=DecisionTreeRegressor(max_depth=3,min_samples_split=5)

cands=[ols,lars,earth,rf]
metas=[ols,lasso,ridge,earth,rf,rtree,gbm]
def helper(X1,y1,X2,y2,cands_bma=cands,cands_sl=cands,metas=metas,gbm=gbm_cv,relative=True):
    
    display(try_super_learners(cands_sl,metas,X1,y1,X2,y2,relative=relative))
#     sl=SuperLearner(cand_learners=cands,V=10,meta_learner=ols).fit(X1,y1)
#     for meta in metas:
#         sl.meta_learner_=meta.fit(sl.Z_train_cv_,y1)
#         df=sl.debug(X1,y1,X2,y2,skip_fit=True)
    display(BMA(cand_learners=cands_bma).debug(X1,y1,X2,y2,relative=relative))
    display(gbm_summary(gbm,X1,y1,X2,y2))


The gprof2dot_magic module is not an IPython extension.


In [2]:
# first simulation study
def sim1(n, seed=seed):
    np.random.seed(seed)
    w=np.random.binomial(1,.4,size=(10,n))
    eps=np.random.normal(0,1,size=n)
    y=2*w[0]*w[9]+4*w[1]*w[6]+3*w[3]*w[4]-\
    5*w[5]*w[9]+3*w[7]*w[8]+w[0]*w[1]*w[3]-\
    2*w[6]*(1-w[5])*w[1]*w[8]-4*(1-w[9])*w[0]*(1-w[3])+eps
    return np.transpose(w),y

train1,test1=sim1(500),sim1(10000)
helper(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,7.2710,9.7481,5.3928
1,LarsCV,7.2710,9.7481,5.3928
2,Earth,1.1998,2.1440,1.0000
3,RandomForestRegressor,1.0000,3.1299,1.6439
4,Meta (LinearRegression),1.0109,1.9675,1.0197
5,Meta (LassoCV),1.0199,1.9829,1.0341
6,Meta (RidgeCV),1.0147,1.9725,1.0285
7,Meta (Earth),1.1131,1.8725,1.0537
8,Meta (RandomForestRegressor),1.1462,1.0000,1.1001
9,Meta (DecisionTreeRegressor),1.3593,1.7339,1.1354


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,7.2710,5.3928,0.0
1,LarsCV,7.2710,5.3928,0.0
2,Earth,1.1998,1.0000,0.0
3,RandomForestRegressor,1.0000,1.6439,1.0
4,BMA,1.0000,1.6439,1.0
5,Min Error,0.8533,1.1969,NaN


                     0
learning_rate     0.01
max_depth         3.00
n_estimators   3000.00


,Learner,Train MSE,Test MSE
0,Gradient Boosting,0.8055,1.2592


In [3]:
# Second simulation (low noise linear)
def sim2(n, noise_ratio=0.1, seed=seed):
    np.random.seed(seed)
    w = np.zeros(12)
    w[0:6] = 0.9
    w[6:8] = 0.4
    w[8:10] = 0.2
    x12 = np.random.poisson(1 ,size=(2,n))
    x36 = np.random.uniform(0,1, size=(4,n))
    x78 = x12*x36[0:2]
    x910 = x36[0:2]*x36[1:3]
    x1112 = np.random.binomial(2, 0.5, size=(2,n))
    y_mat = pd.DataFrame(np.transpose(np.concatenate([x12, x36, x78, x910, x1112], axis=0)))
    X = y_mat[y_mat.columns[[0,1,2,3,4,5,10,11]]]
    Ey = y_mat.multiply(w, axis=1).sum(axis=1)
    var_y = np.var(Ey)
    eps = np.random.normal(0,noise_ratio*var_y,size=n)
    y = Ey + eps
    return X, y
        
train2,test2=sim2(1000,0.1),sim2(10000,0.1)
helper(*train2,*test2)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,1.5753,2.3733,1.3043
1,LarsCV,1.5753,2.3733,1.3043
2,Earth,1.1573,1.8297,1.0030
3,RandomForestRegressor,1.5677,6.2679,3.1943
4,Meta (LinearRegression),1.1658,1.8252,1.0012
5,Meta (LassoCV),1.1595,1.8263,1.0000
6,Meta (RidgeCV),1.1637,1.8259,1.0006
7,Meta (Earth),1.1573,1.8296,1.0032
8,Meta (RandomForestRegressor),1.0000,1.0000,1.1847
9,Meta (DecisionTreeRegressor),2.2970,3.3585,2.2372


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,1.3613,1.3004,0.0
1,LarsCV,1.3613,1.3004,0.0
2,Earth,1.0000,1.0000,1.0
3,RandomForestRegressor,1.3547,3.1847,0.0
4,BMA,1.0000,1.0000,1.0
5,Min Error,0.0809,0.0890,NaN


                     0
learning_rate     0.01
max_depth         3.00
n_estimators   3000.00


,Learner,Train MSE,Test MSE
0,Gradient Boosting,0.0335,0.1376


In [4]:
# third simulation (linear high noise)
train3,test3=sim2(1000,0.35),sim2(10000,0.35)
helper(*train3,*test3)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,1.6789,1.5977,1.0000
1,LarsCV,1.6789,1.5974,1.0000
2,Earth,1.5523,1.6378,1.1678
3,RandomForestRegressor,1.0000,1.9869,1.1910
4,Meta (LinearRegression),1.6045,1.5858,1.0036
5,Meta (LassoCV),1.6039,1.5861,1.0031
6,Meta (RidgeCV),1.5907,1.5860,1.0026
7,Meta (Earth),1.7523,1.5551,367.1754
8,Meta (RandomForestRegressor),1.5497,1.0000,1.0507
9,Meta (DecisionTreeRegressor),1.6917,1.6017,1.1423


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,1.6789,1.0000,0.0
1,LarsCV,1.6789,1.0000,0.0
2,Earth,1.5523,1.1678,0.0
3,RandomForestRegressor,1.0000,1.1910,1.0
4,BMA,1.0000,1.1910,1.0
5,Min Error,0.6109,1.0966,NaN


                     0
learning_rate     0.01
max_depth         3.00
n_estimators   1000.00


,Learner,Train MSE,Test MSE
0,Gradient Boosting,0.6707,1.2009


In [5]:
# non-linear simulation (low noise)
def sim3(n, noise_ratio=0.2, seed=seed):
    np.random.seed(seed)
    x14 = np.random.binomial(1,.4,size=(4,n))
    x48 = np.random.binomial(8, 0.2, size=(4,n))
    x912 = np.random.normal(2, 2, size=(4,n))

    X = np.transpose(pd.DataFrame(np.concatenate([x14,x48,x912])))
    Ey = 0.4*(x48[1]> 3)*(x48[2] < 3) + x14[1]*x14[0]*(4-x48[2])\
        - 0.8*x48[1]*x912[0] + 0.5*x912[3]*((x912[2]>0)*(x912[1]>6)) + x48[1]*(x14[1])\
        + 0.5*x912[1]*(x48[3]>2)*x48[3] + (1-x14[0])*(1+x48[2])

    var_y = np.var(Ey)
    eps = np.random.normal(0,noise_ratio*var_y,size=n)
    y = Ey + eps
    return X, y

train4,test4=sim3(1000,0.1),sim3(10000,0.1)
helper(*train4,*test4)

/home/hurichardj/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hurichardj/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,3.1289,3.7331,2.4434
1,LarsCV,3.1394,3.7367,2.4228
2,Earth,1.2535,1.6959,1.0341
3,RandomForestRegressor,1.0000,2.6850,1.4606
4,Meta (LinearRegression),1.0818,1.6334,1.0010
5,Meta (LassoCV),1.0980,1.6422,1.0000
6,Meta (RidgeCV),1.0824,1.6335,1.0006
7,Meta (Earth),1.0961,1.6421,1.0008
8,Meta (RandomForestRegressor),1.1172,1.0000,1.0660
9,Meta (DecisionTreeRegressor),1.5388,1.7915,1.2460


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,3.1289,2.3627,0.0
1,LarsCV,3.1394,2.3429,0.0
2,Earth,1.2535,1.0000,0.0
3,RandomForestRegressor,1.0000,1.4124,1.0
4,BMA,1.0000,1.4124,1.0
5,Min Error,3.3137,4.8315,NaN


                      0
learning_rate     0.005
max_depth         3.000
n_estimators   3000.000


,Learner,Train MSE,Test MSE
0,Gradient Boosting,2.8295,5.4468


In [6]:
# non-linear simulation (high noise)
train5,test5=sim3(1000,0.35),sim3(10000,0.35)
helper(*train5,*test5)

/home/hurichardj/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,1.8056,1.5489,1.0894
1,LarsCV,1.8150,1.5611,1.0834
2,Earth,1.6116,1.5426,1.0000
3,RandomForestRegressor,1.0000,1.5788,1.0489
4,Meta (LinearRegression),1.4787,1.4613,1.0247
5,Meta (LassoCV),1.4804,1.4616,1.0185
6,Meta (RidgeCV),1.4787,1.4613,1.0245
7,Meta (Earth),1.5788,1.4452,1.0302
8,Meta (RandomForestRegressor),1.6012,1.0000,1.0327
9,Meta (DecisionTreeRegressor),1.7396,1.4388,1.0475


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,1.8056,1.0894,0.0
1,LarsCV,1.8150,1.0834,0.0
2,Earth,1.6116,1.0000,0.0
3,RandomForestRegressor,1.0000,1.0489,1.0
4,BMA,1.0000,1.0489,1.0
5,Min Error,29.7452,50.9260,NaN


                      0
learning_rate     0.005
max_depth         3.000
n_estimators   1000.000


,Learner,Train MSE,Test MSE
0,Gradient Boosting,39.8845,52.8891


In [7]:
diabetes=datasets.load_diabetes()

X_train, X_test, y_train, y_test = train_test_split(
    diabetes.data, diabetes.target, test_size=0.2)

helper(X_train,y_train,X_test,y_test)

/home/hurichardj/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,2.9356,3.0215,1.0000
1,LarsCV,2.9565,3.0102,1.0154
2,Earth,2.5566,3.6122,1.1648
3,RandomForestRegressor,1.0000,3.2875,1.1452
4,Meta (LinearRegression),2.2638,2.9742,1.0065
5,Meta (LassoCV),2.2897,2.9753,1.0072
6,Meta (RidgeCV),2.2638,2.9742,1.0065
7,Meta (Earth),2.9394,2.9978,1.0121
8,Meta (RandomForestRegressor),2.6832,1.1903,1.2078
9,Meta (DecisionTreeRegressor),2.9049,2.6732,1.2033


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,2.9356,1.0000,0.0
1,LarsCV,2.9565,1.0154,0.0
2,Earth,2.5566,1.1648,0.0
3,RandomForestRegressor,1.0000,1.1452,1.0
4,BMA,1.0000,1.1452,1.0
5,Min Error,959.2142,3072.1643,NaN


/home/hurichardj/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


                      0
learning_rate     0.005
max_depth         3.000
n_estimators   1000.000


,Learner,Train MSE,Test MSE
0,Gradient Boosting,1362.2941,3521.4948


In [8]:
pr=pd.read_csv("datasets/CASP.csv")
feature_cols= pr.columns[pr.columns!='RMSD']
from sklearn.preprocessing import scale
pr.loc[:,feature_cols]=scale(pr.loc[:,feature_cols])

/home/hurichardj/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  after removing the cwd from sys.path.


In [9]:
prtrain,prtest=train_test_split(pr.sample(1000))

In [10]:
helper(prtrain.loc[:,feature_cols],prtrain.RMSD,prtest.loc[:,feature_cols],prtest.RMSD)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,2.2860,1.8900,1.0899
1,LarsCV,2.3379,1.8384,1.1367
2,Earth,2.0662,1.9428,1.0350
3,RandomForestRegressor,1.0000,1.8436,1.0239
4,Meta (LinearRegression),1.5869,1.7270,1.0280
5,Meta (LassoCV),1.5509,1.7321,1.0000
6,Meta (RidgeCV),1.5808,1.7271,1.0243
7,Meta (Earth),1.6374,1.6807,1.0104
8,Meta (RandomForestRegressor),1.6905,1.0199,1.0150
9,Meta (DecisionTreeRegressor),1.5560,1.6630,1.0122


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,2.2860,1.0645,0.0
1,LarsCV,2.3379,1.1102,0.0
2,Earth,2.0662,1.0109,0.0
3,RandomForestRegressor,1.0000,1.0000,1.0
4,BMA,1.0000,1.0000,1.0
5,Min Error,11.5377,28.0040,NaN


                      0
learning_rate     0.005
max_depth         3.000
n_estimators   1000.000


,Learner,Train MSE,Test MSE
0,Gradient Boosting,17.51,29.0965


In [11]:
rf1 = ensemble.RandomForestRegressor(n_estimators=500, max_depth=3,
                                    min_samples_split=5, random_state=1)
rf2 = ensemble.RandomForestRegressor(n_estimators=500, max_depth=5,
                                    min_samples_split=5, random_state=1)
rf3 = ensemble.RandomForestRegressor(n_estimators=500, max_depth=7,
                                    min_samples_split=5, random_state=1)
rf4 = ensemble.RandomForestRegressor(n_estimators=500, max_features = 0.5, max_depth=8,
                                    min_samples_split=5, random_state=1)
rf5 = ensemble.RandomForestRegressor(n_estimators=500, max_features = 0.5, max_depth=9,
                                    min_samples_split=5, random_state=1)
helper(*train4,*test4, cands_bma=[rf1,rf2,rf3,rf4,rf5],cands_sl=[rf1,rf2,rf3,rf4,rf5])

,Learner,Train MSE,Train CV MSE,Test MSE
0,RandomForestRegressor,5.3142,3.4907,1.9083
1,RandomForestRegressor,2.9053,2.5342,1.2876
2,RandomForestRegressor,1.6495,2.2230,1.1140
3,RandomForestRegressor,1.4746,2.2398,1.1352
4,RandomForestRegressor,1.1903,2.2151,1.1118
5,Meta (LinearRegression),1.1204,1.8644,1.0075
6,Meta (LassoCV),1.0000,1.8677,1.0000
7,Meta (RidgeCV),1.0596,1.8656,1.0035
8,Meta (Earth),2.3029,1.6823,1.0959
9,Meta (RandomForestRegressor),1.2748,1.0000,1.0709


,Learner,Train MSE,Test MSE,Coefs
0,RandomForestRegressor,4.4647,1.7164,0.0
1,RandomForestRegressor,2.4409,1.1581,0.0
2,RandomForestRegressor,1.3858,1.0020,0.0
3,RandomForestRegressor,1.2389,1.0210,0.0
4,RandomForestRegressor,1.0000,1.0000,1.0
5,BMA,1.0000,1.0000,1.0
6,Min Error,2.4050,6.7900,NaN


                      0
learning_rate     0.005
max_depth         3.000
n_estimators   3000.000


,Learner,Train MSE,Test MSE
0,Gradient Boosting,2.8295,5.4486
